#<font color=yellow>7º Algoritmo Para 5 Espécies de Flores

#Alterações Realizadas:

#Monitoramento de Gradientes:

* Foi adicionado a classe GradientLogger, que calcula e exibe os gradientes para cada época de treinamento.

#Redução de Épocas:

* Para a análise do gradiente, foi reduzido o número de épocas para 10. Isso é feito para evitar sobrecarregar a memória e manter a análise de gradiente mais prática.

#Função de Gradientes:

* A função GradientLogger usa keras.backend.function para calcular os gradientes com base nos dados de treinamento.

##Nota - Com o emprego do cálculo do Gradiente, o custo computacional foi muito alto, por este motivo a diminuição do número de épocas, porém ultrapassou os recursos computacionais não permitindo o resultado final do cálculo dos gradientes da rede neural artificial CNN.

In [ ]:
import numpy as np
from tensorflow import keras
from google.colab import files
import io
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    balanced_accuracy_score,
    precision_score,
    f1_score,
    recall_score,
    roc_curve,
    auc,
    precision_recall_curve,
)

# Defina as dimensões das imagens
img_height = 100
img_width = 100
num_classes = 5  # Cinco classes para diferentes tipos de flores

# Dicionário de rótulos para cada tipo de flor
flower_labels = {
    "Margarida": 0,
    "Dente de Leão": 1,
    "Rosa": 2,
    "Girassol": 3,
    "Tulipa": 4,
}

def carregar_imagens(flower_type):
    print(f"Escolha as Imagens de {flower_type} Para o Treinamento da Rede Neural:")
    images = files.upload()

    data = []
    labels = []
    for filename, content in images.items():
        img = keras.preprocessing.image.load_img(io.BytesIO(content), target_size=(img_height, img_width))
        img_array = keras.preprocessing.image.img_to_array(img)
        data.append(img_array)
        labels.append(flower_labels[flower_type])

    return np.array(data), np.array(labels)

# Carregar e processar as imagens para treinamento
flower_types = ["Margarida", "Dente de Leão", "Rosa", "Girassol", "Tulipa"]

train_data = []
train_labels = []
val_data = []
val_labels = []

for flower_type in flower_types:
    data, labels = carregar_imagens(flower_type)
    train_data_type, val_data_type, train_labels_type, val_labels_type = train_test_split(data, labels, test_size=0.2)
    train_data.append(train_data_type)
    train_labels.append(train_labels_type)
    val_data.append(val_data_type)
    val_labels.append(val_labels_type)

train_data = np.concatenate(train_data, axis=0)
train_labels = np.concatenate(train_labels, axis=0)
val_data = np.concatenate(val_data, axis=0)
val_labels = np.concatenate(val_labels, axis=0)

# Normalizar os dados de imagem
train_data = train_data / 255.0
val_data = val_data / 255.0

# ********************* IMPLEMENTAÇÃO DA REDE NEURAL ARTIFICIAL CNN LENET-5 ***********************

# Construir o modelo LeNet-5 adaptado
model = keras.Sequential([
    keras.layers.Conv2D(
        32,
        kernel_size=(3, 3),
        strides=(1, 1),
        activation="relu",
        input_shape=(img_height, img_width, 3),
        kernel_regularizer=keras.regularizers.l2(0.01),
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(
        64,
        kernel_size=(3, 3),
        strides=(1, 1),
        activation="relu",
        kernel_regularizer=keras.regularizers.l2(0.01),
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(
        128,
        kernel_size=(3, 3),
        strides=(1, 1),
        activation="relu",
        kernel_regularizer=keras.regularizers.l2(0.01),
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(
        256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)
    ),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(
        128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)
    ),
    keras.layers.Dropout(0.5),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(
        64, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)
    ),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(
        32, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)
    ),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(num_classes, activation="softmax"),
])

# Compilar o modelo
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# ********************* FIM DO BLOCO DE CÓDIGO COM A REDE NEURAL CNN***************************************************



#********************************************** Cálculo dos Gradiente **********************************************************************

# Função para monitorar gradientes
class GradientLogger(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Obter os gradientes do modelo
        weights = model.trainable_weights
        gradients = model.optimizer.get_gradients(model.total_loss, weights)

        # Criação de uma sessão Keras para calcular os gradientes
        input_tensors = [model.inputs[0],  # entrada da imagem
                         model.sample_weights[0],  # amostras
                         model.targets[0],  # rótulos
                         keras.backend.learning_phase()]  # fase de aprendizado

        get_gradients = keras.backend.function(inputs=input_tensors, outputs=gradients)

        # Dados de treinamento
        inputs = [train_data,  # dados de entrada
                  np.ones(train_data.shape[0]),  # pesos de amostra
                  train_labels,  # rótulos
                  0]  # fase de aprendizado (0 para avaliação/predição)

        calculated_gradients = get_gradients(inputs)

        # Plotar os gradientes
        plt.figure(figsize=(12, 6))
        for i, gradient in enumerate(calculated_gradients):
            if len(gradient.shape) > 1:  # Evitar os vieses
                plt.subplot(1, len(calculated_gradients), i+1)
                plt.title(f'Gradiente {i+1}')
                plt.imshow(gradient.mean(axis=-1), cmap='viridis', aspect='auto')
                plt.colorbar()
        plt.tight_layout()
        plt.show()

# Treinar o modelo com monitoramento de gradientes
history = model.fit(train_data, train_labels,
                    validation_data=(val_data, val_labels),
                    epochs=10,  # para análise de gradiente, reduzimos o número de épocas
                    batch_size=32,
                    callbacks=[GradientLogger()])

#********************************************** Final do Processo de Cálculo dos Gradientes *************************************************************

# Carregar e processar as imagens para teste
test_data = []
test_labels = []

for flower_type in flower_types:
    print(f"Escolha as Imagens de {flower_type} Para Teste da Rede Neural:")
    images = files.upload()
    data = []
    labels = []
    for filename, content in images.items():
        img = keras.preprocessing.image.load_img(io.BytesIO(content), target_size=(img_height, img_width))
        img_array = keras.preprocessing.image.img_to_array(img)
        data.append(img_array)
        labels.append(flower_labels[flower_type])
    test_data.append(np.array(data))
    test_labels.append(np.array(labels))

test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Normalizar os dados de imagem
test_data = test_data / 255.0

# Avaliar o modelo nos dados de teste
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Prever as classes para os dados de teste
test_predictions = model.predict(test_data)
test_predictions_classes = np.argmax(test_predictions, axis=1)

# Salvar o modelo na pasta de downloads
model.save("/content/downloads/flower_classification_model.h5")

# Plotando gráfico de matriz de confusão para imagens de teste
confusion = confusion_matrix(test_labels, test_predictions_classes)
plt.figure(figsize=(6, 6))
sns.heatmap(
    confusion,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=flower_types,
    yticklabels=flower_types,
)
plt.xlabel("Predito")
plt.ylabel("Atual")
plt.title("Matriz de Confusão para Imagens de Teste")
plt.show()

# Calculando métricas de validação para imagens de teste
accuracy = accuracy_score(test_labels, test_predictions_classes)
balanced_accuracy = balanced_accuracy_score(test_labels, test_predictions_classes)
precision = precision_score(test_labels, test_predictions_classes, average='weighted')
f1 = f1_score(test_labels, test_predictions_classes, average='weighted')
recall = recall_score(test_labels, test_predictions_classes, average='weighted')

print("Acurácia para imagens de teste:", accuracy)
print("Acurácia Balanceada para imagens de teste:", balanced_accuracy)
print("Precisão para imagens de teste:", precision)
print("F1 Score para imagens de teste:", f1)
print("Recall para imagens de teste:", recall)

# Plotar gráficos de perda e acurácia
plt.figure(figsize=(18, 10))

# Gráfico de perda
plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Gráfico de acurácia
plt.subplot(2, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Transformar o problema em binário para uma abordagem "one-vs-rest"
for class_index in range(num_classes):
    print(f"\nClasse: {flower_types[class_index]}")

    # Extrair rótulos binários para a classe atual
    binary_labels = (test_labels == class_index).astype(int)

    # Calcular probabilidades previstas para a classe atual
    predicted_probabilities = test_predictions[:, class_index]

    # Curva Precision-Recall
    precision, recall, thresholds = precision_recall_curve(binary_labels, predicted_probabilities)

    # F1-score para diferentes limiares de probabilidade
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)

    # Limiar que maximiza o F1-score
    optimal_threshold_index = np.argmax(f1_scores)

    # Limiar ótimo
    optimal_threshold = thresholds[optimal_threshold_index]
    optimal_precision = precision[optimal_threshold_index]
    optimal_recall = recall[optimal_threshold_index]

    print("Optimal Threshold that maximizes F1-score (i.e., optimizing both Precision and Recall):", optimal_threshold)
    print("Optimal Precision:", optimal_precision)
    print("Optimal Recall:", optimal_recall)

    # Plotando as curvas
    plt.figure(figsize=(18, 10))

    # Plotando Precision-Recall curve
    plt.subplot(2, 3, 1)
    plt.plot(recall, precision, label='Precision-Recall Curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve for {flower_types[class_index]}')
    plt.axhline(y=optimal_precision, color='blue', linestyle='--', label=f'Optimal Precision ({optimal_precision:.3f})')
    plt.axvline(x=optimal_recall, color='green', linestyle='--', label=f'Optimal Recall ({optimal_recall:.3f})')
    plt.legend()

    # Plotando Precision e Recall VS threshold
    plt.subplot(2, 3, 2)
    plt.plot(thresholds, precision[:-1], label='Precision', color='blue')
    plt.plot(thresholds, recall[:-1], label='Recall', color='green')
    plt.xlabel('Threshold')
    plt.ylabel('Value')
    plt.title(f'Precision and Recall vs. Threshold for {flower_types[class_index]}')
    plt.axvline(x=optimal_threshold, color='red', linestyle='--', label=f'Optimal Threshold ({optimal_threshold:.3f})')
    plt.legend()

    # Plotando F1-score VS threshold
    plt.subplot(2, 3, 3)
    plt.plot(thresholds, f1_scores[:-1], label='F1 Score', color='orange')
    plt.xlabel('Threshold')
    plt.ylabel('F1 Score')
    plt.title(f'F1 Score vs. Threshold for {flower_types[class_index]}')
    plt.axvline(x=optimal_threshold, color='red', linestyle='--', label=f'Optimal Threshold ({optimal_threshold:.3f})')
    plt.legend()

    # Plotando curva ROC e AUC
    plt.subplot(2, 3, 4)
    fpr, tpr, _ = roc_curve(binary_labels, predicted_probabilities)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic (ROC) Curve for {flower_types[class_index]}')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Função para identificar e exibir novas imagens
def identificar_e_exibir_imagens():
    print("Carregue novas imagens para identificação:")
    new_images = files.upload()

    if not new_images:
        print("Nenhuma imagem carregada.")
        return

    num_cols = 5
    num_rows = (len(new_images) + num_cols - 1) // num_cols  # Calcular o número de linhas necessário

    plt.figure(figsize=(10, num_rows * 2))  # Ajustar o tamanho da figura

    for i, (filename, content) in enumerate(new_images.items()):
        img = keras.preprocessing.image.load_img(io.BytesIO(content), target_size=(img_height, img_width))
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalizar e adicionar batch dimension
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction, axis=1)[0]
        confidence = np.max(prediction) * 100

        plt.subplot(num_rows, num_cols, i + 1)
        plt.imshow(img)
        plt.title(f'{flower_types[predicted_class]} ({confidence:.2f}%)')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Função para o loop principal de identificação de imagens
def main_loop():
    while True:
        identificar_e_exibir_imagens()
        action = input("Digite 'novas' para identificar outras imagens ou 'sair' para terminar: ").lower()
        if action == 'sair':
            print("Encerrando o programa.")
            break

# Chamar a função para identificar novas imagens
main_loop()


Escolha as Imagens de Margarida Para o Treinamento da Rede Neural:


Saving Margarida (1).jpg to Margarida (1) (1).jpg
Saving Margarida (2).jpg to Margarida (2) (1).jpg
Saving Margarida (3).jpg to Margarida (3) (1).jpg
Saving Margarida (4).jpg to Margarida (4) (1).jpg
Saving Margarida (5).jpg to Margarida (5) (1).jpg
Saving Margarida (6).jpg to Margarida (6) (1).jpg
Saving Margarida (7).jpg to Margarida (7) (1).jpg
Saving Margarida (8).jpg to Margarida (8) (1).jpg
Saving Margarida (9).jpg to Margarida (9) (1).jpg
Saving Margarida (10).jpg to Margarida (10) (1).jpg
Saving Margarida (11).jpg to Margarida (11) (1).jpg
Saving Margarida (12).jpg to Margarida (12) (1).jpg
Saving Margarida (13).jpg to Margarida (13) (1).jpg
Saving Margarida (14).jpg to Margarida (14) (1).jpg
Saving Margarida (15).jpg to Margarida (15) (1).jpg
Saving Margarida (16).jpg to Margarida (16) (1).jpg
Saving Margarida (17).jpg to Margarida (17) (1).jpg
Saving Margarida (18).jpg to Margarida (18) (1).jpg
Saving Margarida (19).jpg to Margarida (19) (1).jpg
Saving Margarida (20).jpg to M

Saving Dente_de_Leão (1).jpg to Dente_de_Leão (1) (1).jpg
Saving Dente_de_Leão (2).jpg to Dente_de_Leão (2) (1).jpg
Saving Dente_de_Leão (3).jpg to Dente_de_Leão (3) (1).jpg
Saving Dente_de_Leão (4).jpg to Dente_de_Leão (4) (1).jpg
Saving Dente_de_Leão (5).jpg to Dente_de_Leão (5) (1).jpg
Saving Dente_de_Leão (6).jpg to Dente_de_Leão (6) (1).jpg
Saving Dente_de_Leão (7).jpg to Dente_de_Leão (7) (1).jpg
Saving Dente_de_Leão (8).jpg to Dente_de_Leão (8) (1).jpg
Saving Dente_de_Leão (9).jpg to Dente_de_Leão (9) (1).jpg
Saving Dente_de_Leão (10).jpg to Dente_de_Leão (10) (1).jpg
Saving Dente_de_Leão (11).jpg to Dente_de_Leão (11) (1).jpg
Saving Dente_de_Leão (12).jpg to Dente_de_Leão (12) (1).jpg
Saving Dente_de_Leão (13).jpg to Dente_de_Leão (13) (1).jpg
Saving Dente_de_Leão (14).jpg to Dente_de_Leão (14) (1).jpg
Saving Dente_de_Leão (15).jpg to Dente_de_Leão (15) (1).jpg
Saving Dente_de_Leão (16).jpg to Dente_de_Leão (16) (1).jpg
Saving Dente_de_Leão (17).jpg to Dente_de_Leão (17) (1).jp

Saving Rosa (1).jpg to Rosa (1) (1).jpg
Saving Rosa (2).jpg to Rosa (2) (1).jpg
Saving Rosa (3).jpg to Rosa (3) (1).jpg
Saving Rosa (4).jpg to Rosa (4) (1).jpg
Saving Rosa (5).jpg to Rosa (5) (1).jpg
Saving Rosa (6).jpg to Rosa (6) (1).jpg
Saving Rosa (7).jpg to Rosa (7) (1).jpg
Saving Rosa (8).jpg to Rosa (8) (1).jpg
Saving Rosa (9).jpg to Rosa (9) (1).jpg
Saving Rosa (10).jpg to Rosa (10) (1).jpg
Saving Rosa (11).jpg to Rosa (11) (1).jpg
Saving Rosa (12).jpg to Rosa (12) (1).jpg
Saving Rosa (13).jpg to Rosa (13) (1).jpg
Saving Rosa (14).jpg to Rosa (14) (1).jpg
Saving Rosa (15).jpg to Rosa (15) (1).jpg
Saving Rosa (16).jpg to Rosa (16) (1).jpg
Saving Rosa (17).jpg to Rosa (17) (1).jpg
Saving Rosa (18).jpg to Rosa (18) (1).jpg
Saving Rosa (19).jpg to Rosa (19) (1).jpg
Saving Rosa (20).jpg to Rosa (20) (1).jpg
Saving Rosa (21).jpg to Rosa (21) (1).jpg
Saving Rosa (22).jpg to Rosa (22) (1).jpg
Saving Rosa (23).jpg to Rosa (23) (1).jpg
Saving Rosa (24).jpg to Rosa (24) (1).jpg
Saving Ros

Saving Girassol (1).jpeg to Girassol (1) (1).jpeg
Saving Girassol (1).jpg to Girassol (1) (2).jpg
Saving Girassol (2).jpeg to Girassol (2) (1).jpeg
Saving Girassol (2).jpg to Girassol (2) (1).jpg
Saving Girassol (3).jpeg to Girassol (3) (1).jpeg
Saving Girassol (3).jpg to Girassol (3) (1).jpg
Saving Girassol (4).jpeg to Girassol (4) (1).jpeg
Saving Girassol (4).jpg to Girassol (4) (1).jpg
Saving Girassol (5).jpeg to Girassol (5) (1).jpeg
Saving Girassol (5).jpg to Girassol (5) (1).jpg
Saving Girassol (6).jpeg to Girassol (6) (1).jpeg
Saving Girassol (6).jpg to Girassol (6) (1).jpg
Saving Girassol (7).jpeg to Girassol (7) (1).jpeg
Saving Girassol (7).jpg to Girassol (7) (2).jpg
Saving Girassol (8).jpeg to Girassol (8) (1).jpeg
Saving Girassol (8).jpg to Girassol (8) (2).jpg
Saving Girassol (9).jpeg to Girassol (9) (1).jpeg
Saving Girassol (9).jpg to Girassol (9) (2).jpg
Saving Girassol (10).jpeg to Girassol (10) (1).jpeg
Saving Girassol (10).jpg to Girassol (10) (2).jpg
Saving Girassol 

Saving Tulipa (1).jpg to Tulipa (1) (1).jpg
Saving Tulipa (2).jpg to Tulipa (2) (1).jpg
Saving Tulipa (3).jpg to Tulipa (3) (1).jpg
Saving Tulipa (4).jpg to Tulipa (4) (1).jpg
Saving Tulipa (5).jpg to Tulipa (5) (1).jpg
Saving Tulipa (6).jpg to Tulipa (6) (1).jpg
Saving Tulipa (7).jpg to Tulipa (7) (1).jpg
Saving Tulipa (8).jpg to Tulipa (8) (1).jpg
Saving Tulipa (9).jpg to Tulipa (9) (1).jpg
Saving Tulipa (10).jpg to Tulipa (10) (1).jpg
Saving Tulipa (11).jpg to Tulipa (11) (1).jpg
Saving Tulipa (12).jpg to Tulipa (12) (1).jpg
Saving Tulipa (13).jpg to Tulipa (13) (1).jpg
Saving Tulipa (14).jpg to Tulipa (14) (1).jpg
Saving Tulipa (15).jpg to Tulipa (15) (1).jpg
Saving Tulipa (16).jpg to Tulipa (16) (1).jpg
Saving Tulipa (17).jpg to Tulipa (17) (1).jpg
Saving Tulipa (18).jpg to Tulipa (18) (1).jpg
Saving Tulipa (19).jpg to Tulipa (19) (1).jpg
Saving Tulipa (20).jpg to Tulipa (20) (1).jpg
Saving Tulipa (21).jpg to Tulipa (21) (1).jpg
Saving Tulipa (22).jpg to Tulipa (22) (1).jpg
Saving

AttributeError: 'Adam' object has no attribute 'get_gradients'

##NOTA IMPORTANTE: Acima ocorreu o erro pois o cálculo dos gradientes é um processo computacional muito custoso e o processamento em nuvem usado para este Projeto de estudo não comportou o processamento para o cálculo dos gradientes, porém é uma boa prática computacional devido os benefícios do resultado deste cálculo para ajuste fino da Rede Naural Artifical Convolucional CNN.

#Alterações e Correções:

* Substituição de model.optimizer.get_gradients: A função GradientLogger agora usa tf.GradientTape para calcular os gradientes.

* Ajuste no GradientLogger: O GradientLogger foi atualizado para usar tf.GradientTape para monitorar os gradientes após cada época.

* Correções de Código: Alguns ajustes gerais foram feitos para garantir a compatibilidade com TensorFlow 2.x e a atualização de alguns métodos.

##Como Funciona a Atualização de Gradientes:

* A callback GradientLogger usa tf.GradientTape para monitorar os gradientes dos pesos do modelo durante o treinamento e os visualiza após cada época.